In [1]:
import warnings
warnings.filterwarnings("ignore")

# imports best practice pandas
import os

import numpy as np
import pandas as pd
import missingno as msno
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
%matplotlib inline
import math
import pickle
import joblib
import dill
import gzip

import inspect


#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn.feature_selection import VarianceThreshold
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder, RobustScaler, MinMaxScaler
from scipy import stats

# transformers
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import FunctionTransformer


from sklearn.cluster import KMeans

# evaluacion
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
set_config(transform_output = "pandas")

#models
from sklearn.cluster import KMeans

## Funciones Útiles

In [2]:
def report_df(df, verbose = True):
    '''
    Hace un report simple sobre el DataFrame suministrado.
    '''
    print(df.info(verbose = verbose, show_counts=True))
    total_nulos = df.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")

def listarNulos(dataset):
    df_stat = dataset.isnull().sum()[(dataset.isnull().sum()>0)].sort_values(ascending=False).to_frame()
    df_stat.rename(columns={0: 'Nulos'}, inplace = True)
    percent = (df_stat["Nulos"]/dataset.shape[0]).mul(100)
    df_stat["porcentaje"] = percent.round(2)
    return df_stat


## Carga de Datasets

In [3]:
DATA_PATH = "../../data/processed/"
FILE_NAME = "df_capstone_clean.pkl.gz"
PICKLE_PATH = DATA_PATH+FILE_NAME

# Usa gzip.open para abrir el archivo comprimido
with gzip.open(PICKLE_PATH, 'rb') as file:
    df_capstone_clean = pd.read_pickle(file)

print("DataFrame cargado exitosamente")

DataFrame cargado exitosamente


In [4]:
report_df(df_capstone_clean)

df_capstone_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 28 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   pk_cid              5962924 non-null  int64         
 1   pk_partition        5962924 non-null  datetime64[ns]
 2   entry_date          5962924 non-null  datetime64[ns]
 3   entry_channel       5962924 non-null  object        
 4   active_customer     5962924 non-null  int64         
 5   segment             5962924 non-null  object        
 6   short_term_deposit  5962924 non-null  int64         
 7   loans               5962924 non-null  int64         
 8   mortgage            5962924 non-null  int64         
 9   funds               5962924 non-null  int64         
 10  securities          5962924 non-null  int64         
 11  long_term_deposit   5962924 non-null  int64         
 12  em_account_pp       5962924 non-null  int64         
 13  credit_card 

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,age_cat
0,1375586,2018-01-28,2018-01-12,KHL,1,02 - PARTICULARES,0,0,0,0,...,0,0,1,ES,29.0,H,35,0,87218.10,"(30.0, 40.0]"
1,1050611,2018-01-28,2015-08-10,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,13.0,V,23,0,35548.74,"(20.0, 30.0]"
2,1050612,2018-01-28,2015-08-10,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,13.0,V,23,0,122179.11,"(20.0, 30.0]"
3,1050613,2018-01-28,2015-08-10,KHD,0,03 - UNIVERSITARIO,1,0,0,0,...,0,0,0,ES,50.0,H,22,0,119775.54,"(20.0, 30.0]"
4,1050614,2018-01-28,2015-08-10,KHE,1,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,V,23,0,87777.54,"(20.0, 30.0]"


In [5]:
df_capstone_clean_2 = df_capstone_clean.copy()

## Generación de Variables

### "regions_ca_id"


In [6]:
# propuesta de agrupar regiones a comunidades autonomas

mapeo = {1.0: "PV", 2.0: "CM",  3.0: "VC",  4.0: "AN",  5.0: "CL",  6.0: "EX", 7.0: "IB",  8.0: "CT",  9.0: "CL", 10.0: "EX", 11.0: "AN", 
         12.0: "VC", 13.0: "CM", 14.0: "AN", 15.0: "GA", 16.0: "CM", 17.0: "CT", 18.0: "AN", 19.0: "CM", 20.0: "PV", 21.0: "AN", 22.0: "AR", 
         23.0: "AN", 24.0: "CL", 25.0: "CT", 26.0: "RI", 27.0: "GA", 28.0: "MD", 29.0: "AN", 30.0: "MC", 31.0: "NC", 32.0: "GA", 33.0: "AS", 34.0: "CL", 
         35.0: "CN", 36.0: "GA", 37.0: "CL", 38.0: "CN", 39.0: "CB", 40.0: "CL", 41.0: "AN",  42.0: "CL", 43.0: "CT", 44.0: "AR", 45.0: "CM", 46.0: "VC", 
         47.0: "CL", 48.0: "PV",  49.0: "CL", 50.0: "AR", 51.0: "CE", 52.0: "ML"}

In [7]:
df_capstone_clean_2['regions_ca_id'] = df_capstone_clean_2['region_code'].apply(lambda x: mapeo.get(float(x)) if x != 'Extranjero' else x)

In [8]:
print(df_capstone_clean_2['regions_ca_id'].unique())
print('')
print('')
print(f'{df_capstone_clean_2["regions_ca_id"].nunique()} valores únicos')

['AN' 'CM' 'AR' 'CL' 'PV' 'EX' 'CT' 'NC' 'GA' 'MD' 'VC' 'CB' 'IB' 'RI'
 'MC' 'CN' 'AS' 'ML' 'Extranjero' 'CE']


20 valores únicos


In [9]:
df_capstone_clean_2['regions_ca_id'].value_counts(dropna=False)

MD            1185757
AN             953118
CT             749780
VC             598221
GA             538163
CL             402626
CM             307347
MC             296856
EX             205870
AR             166711
CN             150983
AS             134562
CB              69589
PV              67088
IB              62436
RI              34790
NC              31241
CE               2896
ML               2638
Extranjero       2252
Name: regions_ca_id, dtype: int64

In [10]:
df_capstone_clean_2['regions_ca_id'].isnull().sum()

0

### "customer_acquisition_channel"

In [11]:

# Categoría para RED
df_capstone_clean_2.loc[df_capstone_clean_2['entry_channel'] == 'RED', 'customer_acquisition_channel'] = 'RED'

# Categoría para valores numéricos
df_capstone_clean_2.loc[df_capstone_clean_2['entry_channel'].str.isnumeric(), 'customer_acquisition_channel'] = 'Numeric'

# Categorías por prefijo para el resto
df_capstone_clean_2.loc[
    ~df_capstone_clean_2['entry_channel'].isin(['RED']) & 
    ~df_capstone_clean_2['entry_channel'].str.isnumeric(), 'customer_acquisition_channel'
] = df_capstone_clean_2['entry_channel'].str[:2]

In [12]:
df_capstone_clean_2[['customer_acquisition_channel']].value_counts()

customer_acquisition_channel
KH                              4496434
KF                               977585
KA                               423886
RED                               60899
KB                                 1949
Numeric                            1031
KE                                  767
KC                                  229
KD                                   93
KG                                   51
dtype: int64

In [13]:
df_capstone_clean_2[['entry_channel','customer_acquisition_channel']].info()
df_capstone_clean_2[['entry_channel','customer_acquisition_channel']].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 2 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   entry_channel                 object
 1   customer_acquisition_channel  object
dtypes: object(2)
memory usage: 91.0+ MB


,entry_channel,customer_acquisition_channel
0,KHL,KH
1,KHE,KH
2,KHE,KH
3,KHD,KH
4,KHE,KH


In [14]:
df_capstone_clean_2['customer_acquisition_channel'].isnull().sum()

0

### "customer_acquisition_channel_min"

In [15]:
def categorizar_canal_entrada(df):
    # Categoría para RED
    df.loc[df['entry_channel'] == 'RED', 'customer_acquisition_channel_min'] = 'RED'
    
    # Categoría para valores numéricos
    df.loc[df['entry_channel'].str.isnumeric(), 'customer_acquisition_channel_min'] = 'Numeric'
    
    # Categoría general para el resto
    df.loc[
        ~df['entry_channel'].isin(['RED']) & 
        ~df['entry_channel'].str.isnumeric(), 'customer_acquisition_channel_min'
    ] = 'Prefix'

    return df

# Aplicar la función al DataFrame
df_capstone_clean_2 = categorizar_canal_entrada(df_capstone_clean_2)


In [16]:
df_capstone_clean_2["customer_acquisition_channel_min"].value_counts()

Prefix     5900994
RED          60899
Numeric       1031
Name: customer_acquisition_channel_min, dtype: int64

In [17]:
df_capstone_clean_2["customer_acquisition_channel_min"].isnull().sum()

0

### "nr_product"

In [18]:
products_list = ['short_term_deposit', 'loans', 'mortgage', 'funds',
                'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 'pension_plan',
                'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']

df_capstone_clean_2['nr_product'] = df_capstone_clean_2[products_list].sum(axis=1)
# df_capstone_clean_2['producsts_>_1'] = (df_capstone_clean_2['nr_product'] > 0).astype('int')

In [19]:
df_capstone_clean_2["nr_product"].value_counts()

1    3995714
0    1121507
2     528593
3     150269
4     105720
5      42890
6      14809
7       2799
8        573
9         50
Name: nr_product, dtype: int64

In [20]:
df_capstone_clean_2["nr_product"].isnull().sum()

0

### "nr_acccounts", "nr_saving_and_investment", "nr_financing"
Indica cuantos productos relacionados a inversiones tiene el cliente

In [21]:
products_list = ['short_term_deposit', 'loans', 'mortgage', 'funds',
                'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 'pension_plan',
                'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']

accounts_list = ['em_acount','em_account_p','em_account_pp','emc_account']
saving_and_investment_products_list = ['short_term_deposit','long_term_deposit','funds','pension_plan','securities']
financing_products_list = ['loans','mortgage','credit_card','payroll','payroll_account','debit_card']

if len(products_list) - (len(accounts_list) + len(saving_and_investment_products_list) + len(financing_products_list)) == 0:
    print('Todos los productos incluídos en la lista de productos')
else:
    print('Chequear, algunos productos no están catalogados')

Todos los productos incluídos en la lista de productos


In [22]:
df_capstone_clean_2['nr_accounts'] = df_capstone_clean_2[accounts_list].sum(axis='columns')
df_capstone_clean_2['nr_saving_and_investment'] = df_capstone_clean_2[saving_and_investment_products_list].sum(axis='columns')
df_capstone_clean_2['nr_financing'] = df_capstone_clean_2[financing_products_list].sum(axis='columns')

In [23]:
df_capstone_clean_2["nr_accounts"].value_counts()

1    4361685
0    1427783
2     173456
Name: nr_accounts, dtype: int64

In [24]:
df_capstone_clean_2["nr_saving_and_investment"].value_counts()

0    5608901
1     333841
2      18450
3       1581
4        151
Name: nr_saving_and_investment, dtype: int64

In [25]:
df_capstone_clean_2["nr_financing"].value_counts()

0    5236075
1     446731
2     140910
3     115779
4      23200
5        229
Name: nr_financing, dtype: int64

In [26]:
df_capstone_clean_2["nr_accounts"].isnull().sum()

0

In [27]:
df_capstone_clean_2["nr_saving_and_investment"].isnull().sum()

0

In [28]:
df_capstone_clean_2["nr_financing"].isnull().sum()

0

### "nr_product_gap_to_max"
Diferencia entre el "nr_product" de la partición y el máximo histórico del cliente

In [29]:
df_capstone_clean_2['nr_product_max'] = df_capstone_clean_2.groupby('pk_cid')['nr_product'].transform(max)
df_capstone_clean_2['nr_product_gap_to_max'] = df_capstone_clean_2['nr_product_max'] - df_capstone_clean_2['nr_product']

In [30]:
df_capstone_clean_2["nr_product_max"].value_counts()

1    3710888
0     969700
2     717400
3     239477
4     207415
5      78759
6      31108
7       6537
8       1453
9        187
Name: nr_product_max, dtype: int64

In [31]:
df_capstone_clean_2["nr_product_gap_to_max"].value_counts()

0    5200669
1     520446
2     179733
3      51531
4       8721
5       1598
6        191
7         35
Name: nr_product_gap_to_max, dtype: int64

In [32]:
df_capstone_clean_2["nr_product_max"].isnull().sum()

0

In [33]:
df_capstone_clean_2["nr_product_gap_to_max"].isnull().sum()

0

### "nr_investment"

In [34]:
financial_products_list = ['funds','pension_plan','securities']
df_capstone_clean_2['nr_investment'] = df_capstone_clean_2[financial_products_list].sum(axis='columns')

In [35]:
df_capstone_clean_2["nr_investment"].value_counts()

0    5710433
1     245384
2       6671
3        436
Name: nr_investment, dtype: int64

In [36]:
df_capstone_clean_2["nr_investment"].isnull().sum()

0

### "product_mix_diversity"
Diversidad en el tipo de productos contratados, para entender la complejidad de las necesidades financieras del cliente.

In [37]:
products_list = ['short_term_deposit', 'loans', 'mortgage', 'funds',
                'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 'pension_plan',
                'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']

accounts_list = ['em_acount','em_account_p','em_account_pp','emc_account']
saving_and_investment_products_list = ['short_term_deposit','long_term_deposit','funds','pension_plan','securities']
financing_products_list = ['loans','mortgage','credit_card','payroll','payroll_account','debit_card']

In [38]:
df_capstone_clean_2['product_mix_diversity'] = (df_capstone_clean_2['nr_accounts'].apply(lambda x: x/len(accounts_list)) + \
                                               df_capstone_clean_2['nr_saving_and_investment'].apply(lambda x: x/len(saving_and_investment_products_list)) + \
                                               df_capstone_clean_2['nr_financing'].apply(lambda x: x/len(financing_products_list)))\
                                               .apply(lambda x: x/3)

In [39]:
df_capstone_clean_2["product_mix_diversity"].value_counts()

0.083333    3893549
0.000000    1121507
0.138889     313848
0.166667     106573
0.233333      97621
             ...   
0.544444          5
0.200000          3
0.305556          3
0.405556          2
0.377778          1
Name: product_mix_diversity, Length: 65, dtype: int64

In [40]:
df_capstone_clean_2["product_mix_diversity"].isnull().sum()

0

### "salary_CA"

In [41]:
# Suponiendo que df_capstone_clean_2 es tu DataFrame y que ya tiene una columna "regions_ca_id"
# con los códigos de las comunidades autónomas.

# Mapeo de las comunidades autónomas a su renta media correspondiente
renta_media = {
    "AN": 11977, "AR": 16081, "AS": 16081, "IB": 16492, "CN": 13076, "CB": 15332,
    "CM": 15129, "CL": 15129, "CT": 17459, "VC": 13643, "EX": 11697, "GA": 14438,
    "MD": 19481, "MC": 12300, "NC": 18245, "PV": 18824, "RI": 15513, "CE": 13368,
    "ML": 12676
}


# Crear la nueva columna "salary_CA" mapeando los valores de "regions_ca_id" 
# a través del diccionario de renta media
df_capstone_clean_2['salary_CA'] = df_capstone_clean_2['regions_ca_id'].map(renta_media)

# Aplicar el salario medio calculado a la nueva columna 'salary_CA'
df_capstone_clean_2['salary_CA'] = df_capstone_clean_2['regions_ca_id'].apply(lambda x: renta_media.get(x, 15102.16))


# Mostrar el DataFrame resultante para verificar
print(df_capstone_clean_2)

          pk_cid pk_partition entry_date entry_channel  active_customer  \
0        1375586   2018-01-28 2018-01-12           KHL                1   
1        1050611   2018-01-28 2015-08-10           KHE                0   
2        1050612   2018-01-28 2015-08-10           KHE                0   
3        1050613   2018-01-28 2015-08-10           KHD                0   
4        1050614   2018-01-28 2015-08-10           KHE                1   
...          ...          ...        ...           ...              ...   
5962919  1166765   2019-05-28 2016-08-14           KHE                0   
5962920  1166764   2019-05-28 2016-08-14           KHE                0   
5962921  1166763   2019-05-28 2016-08-14           KHE                1   
5962922  1166789   2019-05-28 2016-08-14           KHE                0   
5962923  1550586   2019-05-28 2019-05-13           KFC                1   

                    segment  short_term_deposit  loans  mortgage  funds  ...  \
0         02 - PART

In [42]:
df_capstone_clean_2["salary_CA"].value_counts()

19481.00    1185757
11977.00     953118
17459.00     749780
15129.00     709973
13643.00     598221
14438.00     538163
16081.00     301273
12300.00     296856
11697.00     205870
13076.00     150983
15332.00      69589
18824.00      67088
16492.00      62436
15513.00      34790
18245.00      31241
13368.00       2896
12676.00       2638
15102.16       2252
Name: salary_CA, dtype: int64

In [43]:
df_capstone_clean_2["salary_CA"].isnull().sum()

0

### "customer_tenure"
Tiempo desde entry_date hasta la fecha actual, para medir la lealtad y duración de la relación.

In [44]:
# CUSTOMER TENURE
# Tiempo desde entry_date hasta la fecha actual, para medir la lealtad y duración de la relación.

MIN_DATE_entry_date = df_capstone_clean_2["entry_date"].min()
MAX_DATE_entry_date = df_capstone_clean_2["entry_date"].max()

MIN_DATE_pk_partition = df_capstone_clean_2["pk_partition"].min()
MAX_DATE_pk_partition  =df_capstone_clean_2["pk_partition"].max()

df_capstone_clean_2['last_partition'] = df_capstone_clean_2["pk_partition"].max()

df_capstone_clean_2["customer_tenure"] = df_capstone_clean_2['last_partition'] - df_capstone_clean_2['entry_date']
df_capstone_clean_2['customer_tenure'] = df_capstone_clean_2['customer_tenure'].dt.days

df_capstone_clean_2.drop('last_partition', axis=1, inplace=True)

print(f"Min date entry is {MIN_DATE_entry_date}\nMax date entry is {MAX_DATE_entry_date}")
print(f"Min date pk_partition is {MIN_DATE_pk_partition}\nMax date pk_partition is {MAX_DATE_pk_partition}")
print(df_capstone_clean_2.sample(10))

Min date entry is 2015-01-01 00:00:00
Max date entry is 2019-05-31 00:00:00
Min date pk_partition is 2018-01-28 00:00:00
Max date pk_partition is 2019-05-28 00:00:00
          pk_cid pk_partition entry_date entry_channel  active_customer  \
5644400  1504320   2019-05-28 2018-11-16           KHQ                0   
1684926  1275696   2018-07-28 2017-07-23           KHE                0   
3280829  1064764   2018-11-28 2015-09-21           KHE                1   
192918   1157765   2018-01-28 2016-08-06           KHE                1   
4466590  1419480   2019-02-28 2018-07-28           KHQ                1   
5899968  1241670   2019-05-28 2017-02-04           KHE                0   
1639525  1333838   2018-07-28 2017-10-15           KHE                0   
4306330  1223257   2019-02-28 2016-11-21           KFC                0   
2898720  1336230   2018-10-28 2017-10-20           KHE                0   
1978666  1339287   2018-08-28 2017-10-22           KHE                0   

        

In [45]:
df_capstone_clean_2["customer_tenure"].value_counts()

669     57389
602     54287
662     45746
956     40804
1028    33414
        ...  
1490       11
1077       11
1378       11
362         9
757         7
Name: customer_tenure, Length: 1497, dtype: int64

In [46]:
df_capstone_clean_2["customer_tenure"].isnull().sum()

0

### "digital_interaction_index"
Un índice que mide la preferencia del cliente por el canal digital frente a canales más tradicionales. Esto se puede inferir del canal de entrada y la adopción de productos digitales como em_account, em_account_p, em_account_pp, y emc_account

In [47]:
# DIGITAL INTERACTION 
# "digital_interaction_index"
# Un índice que mide la preferencia del cliente por el canal digital frente a canales más tradicionales. 

# Lista de productos digitales y total de productos
accounts_list = ['em_acount', 'em_account_p', 'em_account_pp', 'emc_account']
products_list = ['short_term_deposit', 'loans', 'mortgage', 'funds',
                 'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 
                 'payroll', 'pension_plan', 'payroll_account', 'emc_account', 
                 'debit_card', 'em_account_p', 'em_acount']

# Calcular el número acumulado de productos digitales y el total de productos contratados por cliente
df_capstone_clean_2['digital_products_count'] = df_capstone_clean_2[accounts_list].sum(axis=1)
df_capstone_clean_2['total_products_count'] = df_capstone_clean_2[products_list].sum(axis=1)

# Asegurar que el denominador no sea cero para evitar división por cero
df_capstone_clean_2['total_products_count'] = df_capstone_clean_2['total_products_count'].apply(lambda x: max(x, 1))

# Calcular el Digital Interaction Index directamente en df_capstone_clean_2
df_capstone_clean_2['digital_interaction_index'] = df_capstone_clean_2['digital_products_count'] / df_capstone_clean_2['total_products_count']

df_capstone_clean_2.drop('digital_products_count', axis=1, inplace=True)
df_capstone_clean_2.drop('total_products_count', axis=1, inplace=True)
# Mostrar un sample del DataFrame actualizado para verificar la nueva columna
print(df_capstone_clean_2[['pk_cid', 'digital_interaction_index']].sample(10))

          pk_cid  digital_interaction_index
838284   1359132                        1.0
4582446  1199282                        1.0
1466426  1225482                        1.0
1687406  1256101                        1.0
1626509  1034042                        1.0
3394190  1077446                        1.0
2960903  1429897                        1.0
5393315  1452365                        1.0
5563064  1411469                        1.0
907146   1079563                        0.5


In [48]:
df_capstone_clean_2["digital_interaction_index"].value_counts()

1.000000    3996392
0.000000    1427783
0.500000     390302
0.666667      52695
0.333333      34717
0.200000      26882
0.250000      16322
0.166667      11776
0.400000       2944
0.142857       1594
0.285714       1141
0.125000        326
0.111111         30
0.222222         20
Name: digital_interaction_index, dtype: int64

In [49]:
df_capstone_clean_2["digital_interaction_index"].isnull().sum()

0

### "financial_health_score"

Una puntuación compuesta basada en el ratio de ingresos frente a productos financieros de alto compromiso (como hipotecas y préstamos) y la diversidad de productos de ahorro. Ayuda a identificar clientes con buena salud financiera que podrían ser buenos candidatos para ofertas de productos de inversión.

In [50]:
# FINANCIAL HEALTH
# "financial_health_score"
# Una puntuación compuesta basada en el ratio de ingresos frente a productos financieros de alto compromiso (como hipotecas y préstamos) y la diversidad de productos de ahorro. Ayuda a identificar clientes con buena salud financiera que podrían ser buenos candidatos para ofertas de productos de inversión.

# Asegurar que las columnas utilizadas para cálculos no contengan NaNs, reemplazándolos con 0 donde sea lógico
df_capstone_clean_2.fillna({'mortgage': 0, 'loans': 0, 'salary': 0, 'em_acount': 0, 'long_term_deposit': 0, 'pension_plan': 0}, inplace=True)

# Calcular el total de compromisos financieros
df_capstone_clean_2['financial_commitments'] = df_capstone_clean_2['mortgage'] + df_capstone_clean_2['loans']  # Añade más columnas si hay más tipos de compromisos financieros

# Asegurar que el denominador en el cálculo del ratio no sea cero
df_capstone_clean_2['financial_commitments'] = df_capstone_clean_2['financial_commitments'].apply(lambda x: np.where(x == 0, 1, x))

# Ratio de Ingresos frente a Compromisos Financieros, maneja el caso de 'salary' siendo 0 asignando un valor mínimo
df_capstone_clean_2['income_to_commitment_ratio'] = df_capstone_clean_2['salary'] / df_capstone_clean_2['financial_commitments']

# Calcular la diversidad de productos de ahorro contratados por el cliente
df_capstone_clean_2['savings_product_diversity'] = df_capstone_clean_2[['short_term_deposit', 'long_term_deposit', 'pension_plan']].sum(axis=1)

# Normalizar los valores para el cálculo del score
max_income_ratio = df_capstone_clean_2['income_to_commitment_ratio'].max() if df_capstone_clean_2['income_to_commitment_ratio'].max() != 0 else 1
max_savings_diversity = df_capstone_clean_2['savings_product_diversity'].max() if df_capstone_clean_2['savings_product_diversity'].max() != 0 else 1

df_capstone_clean_2['normalized_income_ratio'] = df_capstone_clean_2['income_to_commitment_ratio'] / max_income_ratio
df_capstone_clean_2['normalized_savings_diversity'] = df_capstone_clean_2['savings_product_diversity'] / max_savings_diversity

# Calcular el Financial Health Score como un promedio de los dos valores normalizados
df_capstone_clean_2['financial_health_score'] = (df_capstone_clean_2['normalized_income_ratio'] + df_capstone_clean_2['normalized_savings_diversity']) / 2

# Asegurarse de que financial_health_score no contenga NaN
df_capstone_clean_2['financial_health_score'].fillna(0, inplace=True)

df_capstone_clean_2.drop('financial_commitments', axis=1, inplace=True)
df_capstone_clean_2.drop('income_to_commitment_ratio', axis=1, inplace=True)
df_capstone_clean_2.drop('savings_product_diversity', axis=1, inplace=True)
df_capstone_clean_2.drop('normalized_income_ratio', axis=1, inplace=True)
df_capstone_clean_2.drop('normalized_savings_diversity', axis=1, inplace=True)

print(df_capstone_clean_2[['pk_cid', 'financial_health_score']].sample(10))

          pk_cid  financial_health_score
331694   1317058                0.001519
845212   1363331                0.001519
1288898  1082197                0.001474
537803   1149241                0.001519
2163384  1174423                0.001519
4480835    72422                0.003140
4881890  1541507                0.001671
4654807  1431798                0.000787
5923262  1114008                0.001447
37324    1035098                0.000838


In [51]:
df_capstone_clean_2["financial_health_score"].value_counts()

0.001519    781761
0.001542    192666
0.001465    161792
0.001554    109049
0.001442     57438
             ...  
0.000737         1
0.167789         1
0.167628         1
0.000731         1
0.169421         1
Name: financial_health_score, Length: 284858, dtype: int64

In [52]:
df_capstone_clean_2["financial_health_score"].isnull().sum()

0

### "entry_channel_salary"
Analizar cómo el canal de entrada se correlaciona con el rango de salario del cliente, para identificar si ciertos canales atraen a clientes con diferentes niveles de ingresos.

In [53]:
# ENTRY CHANNEL & SALARY CORRELATION

# "entry_channel_salary"
# Analizar cómo el canal de entrada se correlaciona con el rango de salario del cliente, para identificar si ciertos canales atraen a clientes con diferentes niveles de ingresos.


# Calcular el salario medio por canal de entrada
mean_salary_by_channel = df_capstone_clean_2.groupby('entry_channel')['salary'].mean()

# Crear una nueva columna en df_capstone_clean_2 que mapea el salario medio calculado de vuelta a cada fila basándose en su canal de entrada
df_capstone_clean_2['entry_channel_salary'] = df_capstone_clean_2['entry_channel'].map(mean_salary_by_channel)

# Si prefieres usar la mediana en lugar de la media, debido a su resistencia a los valores atípicos, simplemente reemplaza .mean() por .median()

# Verificar los resultados
print(df_capstone_clean_2[['pk_cid', 'entry_channel', 'salary', 'entry_channel_salary']].sample(10))

          pk_cid entry_channel      salary  entry_channel_salary
4717447  1527173           KFC  105618.105         112099.210555
5261868  1023923           KHE   72876.840         108529.905617
1909868  1289985           KHE   89127.570         108529.905617
5006616  1001994           KFC   89810.520         112099.210555
473350   1050419           KHE   89810.520         108529.905617
5184825  1082638           KHE   82570.410         108529.905617
2185778  1245284           KFC  143429.190         112099.210555
2578738  1096600           KHE  140550.780         108529.905617
192659   1157932           KHE   87777.540         108529.905617
600225   1288100           KHE  126806.010         108529.905617


In [54]:
df_capstone_clean_2["entry_channel_salary"].value_counts()

108529.905617    3116090
112099.210555     898070
97521.064767      666987
117248.403277     416522
110141.452954     231300
                  ...   
121974.000000         11
91156.260000          10
170212.140000          8
118381.380000          2
96539.220000           2
Name: entry_channel_salary, Length: 65, dtype: int64

In [55]:
df_capstone_clean_2["entry_channel_salary"].isnull().sum()

0

### "entry_channel_loyalty"

Medir la lealtad del cliente (por ejemplo, a través de la duración de la relación o la cantidad de productos contratados) en función del canal de entrada, para identificar qué canales contribuyen a una mayor lealtad del cliente.

In [56]:
# ENTRY CHANNEL AND CUSTOMER LOYALTY
# "entry_channel_loyalty"
#  Medir la lealtad del cliente (por ejemplo, a través de la duración de la relación o la cantidad de productos contratados) en función del canal de entrada, para identificar qué canales contribuyen a una mayor lealtad del cliente.

# Asegúrate de que las fechas estén en formato datetime si aún no lo están
df_capstone_clean_2['entry_date'] = pd.to_datetime(df_capstone_clean_2['entry_date'])
df_capstone_clean_2['pk_partition'] = pd.to_datetime(df_capstone_clean_2['pk_partition'])

# Filtrar la última partición
df_capstone_clean_2_last_partition = df_capstone_clean_2[df_capstone_clean_2['pk_partition'] == '2019-05-28']

# Calcular la duración de la relación solo para fechas de entrada válidas
df_capstone_clean_2['relationship_duration'] = np.where(
    df_capstone_clean_2['entry_date'] <= df_capstone_clean_2_last_partition['pk_partition'].max(),
    (df_capstone_clean_2_last_partition['pk_partition'].max() - df_capstone_clean_2['entry_date']).dt.days,
    np.nan  # Asignar NaN si la fecha de entrada es inválida
)

# Calcular la duración media de la relación por canal de entrada
mean_loyalty_by_channel = df_capstone_clean_2.groupby('entry_channel')['relationship_duration'].mean()

# Mapear la duración media de la relación (lealtad) de vuelta a df_capstone_clean_2
df_capstone_clean_2['entry_channel_loyalty'] = df_capstone_clean_2['entry_channel'].map(mean_loyalty_by_channel)

# Opcional: puedes decidir llenar los valores NaN con un valor por defecto, por ejemplo, 0 o la media general
# df_capstone_clean_2['entry_channel_loyalty'].fillna(0, inplace=True)
# o
# df_capstone_clean_2['entry_channel_loyalty'].fillna(df_capstone_clean_2['entry_channel_loyalty'].mean(), inplace=True)

df_capstone_clean_2.drop('relationship_duration', axis=1, inplace=True)

# Verificación: mostrar un sample del DataFrame actualizado
print(df_capstone_clean_2[['pk_cid', 'entry_channel', 'entry_channel_loyalty']].sample(10))



          pk_cid entry_channel  entry_channel_loyalty
2470978  1369123           KHK             486.337260
4649774  1445043           KHQ             255.868939
610736   1273814           KHE             975.383414
2451174  1398384           KHK             486.337260
1970416  1360761           KHK             486.337260
3121365  1193686           KHE             975.383414
2848159  1288862           KHE             975.383414
941538   1045693           KHE             975.383414
1875901  1321743           KHE             975.383414
471965   1069290           KHE             975.383414


In [57]:
df_capstone_clean_2["entry_channel_loyalty"].value_counts()

975.383414     3116090
1003.291133     898070
255.868939      666987
969.855036      416522
486.337260      231300
                ...   
33.200000           10
646.000000           8
318.000000           2
53.000000            2
89.000000            1
Name: entry_channel_loyalty, Length: 68, dtype: int64

In [58]:
df_capstone_clean_2["entry_channel_loyalty"].isnull().sum()

0

### "periods_active"

In [59]:
df_capstone_clean_2['periods_active'] = df_capstone_clean_2.groupby('pk_cid')['active_customer'].transform(sum)

In [60]:
df_capstone_clean_2["periods_active"].value_counts()

0     3286200
17    1562232
9      112832
10     110529
8      103340
16      91189
11      87102
7       80878
15      69333
14      60751
6       57492
5       56372
13      54342
3       53434
4       52429
12      48481
2       39246
1       36742
Name: periods_active, dtype: int64

In [61]:
df_capstone_clean_2["periods_active"].isnull().sum()

0

### "season"

In [62]:
def asignar_temporada(mes):
     if mes.month in [11, 12]:
         return 'High_season'
     elif mes.month in [1, 2]:
         return 'Middle_season'
     else:
         return 'Low_Season'

df_capstone_clean_2['season'] = df_capstone_clean_2['entry_date'].apply(asignar_temporada)

In [63]:
df_capstone_clean_2['season'].value_counts()

Low_Season       4472421
High_season      1056581
Middle_season     433922
Name: season, dtype: int64

In [64]:
df_capstone_clean_2['season'].isnull().sum()

0

### "entry_date_month"

In [65]:
df_capstone_clean_2["entry_date_month"] = df_capstone_clean_2["entry_date"].dt.month

In [66]:
df_capstone_clean_2['entry_date_month'].value_counts()

10    1250335
9      920300
7      855554
8      775642
11     720107
12     336474
1      238367
2      195555
3      178373
4      170199
5      168349
6      153669
Name: entry_date_month, dtype: int64

In [67]:
df_capstone_clean_2['entry_date_month'].isnull().sum()

0

### "entry_date_year"

In [68]:
df_capstone_clean_2["entry_date_year"] = df_capstone_clean_2["entry_date"].dt.year

In [69]:
df_capstone_clean_2['entry_date_year'].value_counts()

2017    1725466
2016    1515258
2015    1466793
2018    1171826
2019      83581
Name: entry_date_year, dtype: int64

In [70]:
df_capstone_clean_2['entry_date_year'].isnull().sum()

0

### "working_day"

In [71]:
# Asumiendo que df_capstone_clean_2 ya existe y tiene una columna 'entry_date'
# No necesitas crear el DataFrame de nuevo si ya existe

# Función para verificar si una fecha es un día laborable (de lunes a viernes)
# y devolver 1 para días laborables y 0 para no laborables
def es_dia_laborable(fecha):
    return int(fecha.weekday() < 5)  # 0 a 4 representan lunes a viernes, convertido a int

# Aplicar la función directamente a la columna "entry_date" del DataFrame original
df_capstone_clean_2['working_day'] = df_capstone_clean_2['entry_date'].apply(es_dia_laborable)


#1 = True
#0 = False

In [72]:
df_capstone_clean_2['working_day'].value_counts()

1    3676059
0    2286865
Name: working_day, dtype: int64

In [73]:
df_capstone_clean_2['working_day'].isnull().sum()

0

### "loan_utilization_index"

In [74]:
# LOAN UTILIZATION
# loan_utilization_index: 
# Uso de préstamos e hipotecas respecto al número total de productos, para identificar la dependencia de financiamiento.

loans_products_list = ['loans', 'mortgage']

df_capstone_clean_2['total_products_loans'] = df_capstone_clean_2[loans_products_list].sum(axis=1)
df_capstone_clean_2['loan_utilization_index'] = df_capstone_clean_2.groupby('pk_cid')['total_products_loans'].transform(max)

df_capstone_clean_2.drop('total_products_loans', axis=1, inplace=True)
print(df_capstone_clean_2.sample(10))

          pk_cid pk_partition entry_date entry_channel  active_customer  \
5547609  1349550   2019-05-28 2017-11-05           KHE                0   
3189114  1384535   2018-11-28 2018-02-24           KHK                1   
3311822  1111883   2018-11-28 2015-12-12           KHE                0   
3229928  1289647   2018-11-28 2017-08-04           KHE                1   
4391892  1527709   2019-02-28 2019-01-26           KHM                0   
5746668  1449530   2019-05-28 2018-09-15           KHQ                0   
5733355  1464815   2019-05-28 2018-09-30           KHQ                0   
2288306  1318911   2018-09-28 2017-09-29           KFC                1   
4092335  1224835   2019-01-28 2016-11-25           KHE                0   
397654   1006854   2018-02-28 2015-02-28           KFA                1   

                    segment  short_term_deposit  loans  mortgage  funds  ...  \
5547609  03 - UNIVERSITARIO                   0      0         0      0  ...   
3189114   02 -

In [75]:
df_capstone_clean_2['loan_utilization_index'].value_counts()

0    5961797
1       1127
Name: loan_utilization_index, dtype: int64

In [76]:
df_capstone_clean_2['loan_utilization_index'].isnull().sum()

0

### "ES_flag"

In [77]:
df_capstone_clean_2['ES_Flag'] = (df_capstone_clean_2['country_id'] == 'ES').astype('int')

In [78]:
df_capstone_clean_2['ES_Flag'].value_counts()

1    5960672
0       2252
Name: ES_Flag, dtype: int64

In [79]:
df_capstone_clean_2['ES_Flag'].isnull().sum()

0

### "client_fidelity"

In [80]:
products_list = ['short_term_deposit', 'loans', 'mortgage', 'funds',
                'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 'pension_plan',
                'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']

df_capstone_clean_2['total_products'] = df_capstone_clean_2[products_list].sum(axis=1)

In [81]:

df_capstone_clean_2['client_fidelity'] = ''

client_losing_fidelity = df_capstone_clean_2['total_products'] < df_capstone_clean_2['nr_product_max']
client_stable = df_capstone_clean_2['total_products'] == df_capstone_clean_2['nr_product_max']
# client_increasing_fidelity = df_capstone_clean_2['total_products'] > df_capstone_clean_2['nr_product_max']

df_capstone_clean_2['client_fidelity'] = np.where(client_losing_fidelity, 'losing_fidelity',df_capstone_clean_2['client_fidelity'])
df_capstone_clean_2['client_fidelity'] = np.where(client_stable, 'stable',df_capstone_clean_2['client_fidelity'])

In [82]:
df_capstone_clean_2['client_fidelity'].value_counts()

stable             5200669
losing_fidelity     762255
Name: client_fidelity, dtype: int64

In [83]:
df_capstone_clean_2['client_fidelity'].isnull().sum()

0

### "nr_debt"

In [84]:
# nr_debt
# Indica cuantos productos relacionados a deuda/debitos tiene el cliente.

debt_products_list = ['loans', 'mortgage', 'credit_card', 'debit_card']
df_capstone_clean_2['total_products_debt'] = df_capstone_clean_2[debt_products_list].sum(axis=1)
df_capstone_clean_2['nr_debt'] = df_capstone_clean_2.groupby('pk_cid')['total_products_debt'].transform(max)

df_capstone_clean_2.drop('total_products_debt', axis=1, inplace=True)
print(df_capstone_clean_2.sample(30))

          pk_cid pk_partition entry_date entry_channel  active_customer  \
3227165  1293301   2018-11-28 2017-08-06           KHE                1   
3461469  1267909   2018-12-28 2017-07-08           KFC                0   
4436089  1374550   2019-02-28 2018-01-07           KHK                1   
828038   1381898   2018-04-28 2018-02-11           KHK                1   
4642000  1141528   2019-02-28 2016-07-17           KHE                1   
274634   1224131   2018-02-28 2016-11-22           KHE                0   
5892947  1230059   2019-05-28 2016-12-09           KFC                1   
3417209  1052294   2018-12-28 2015-08-16           KHE                0   
3533420  1141483   2018-12-28 2016-07-17           KHE                0   
3336966  1052795   2018-11-28 2015-08-20           KFA                1   
3481480  1227807   2018-12-28 2016-12-02           KHE                0   
3986850  1367886   2019-01-28 2017-12-11           KHE                0   
3949340  1389996   2019-0

In [85]:
df_capstone_clean_2['nr_debt'].value_counts()

0    4996752
1     870893
2      94759
3        520
Name: nr_debt, dtype: int64

In [86]:
df_capstone_clean_2['nr_debt'].isnull().sum()

0

### "entry_channel_product"

In [87]:
# ENTRY CHANNEL AND PRODUCT TYPE CORRELATION
# "entry_channel_product"
# Una variable que mide la correlación entre el canal de entrada y tipos específicos de productos contratados (por ejemplo, si ciertos canales tienden a atraer más clientes que contratan depósitos a largo plazo o inversiones).

# Definir la lista de columnas de productos
products_list = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 
                 'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 
                 'pension_plan', 'payroll_account', 'emc_account', 'debit_card', 
                 'em_account_p', 'em_acount']

# Paso 1: Sumar los productos contratados por cliente a través de todas las particiones
# Crear un DataFrame temporal que sume los productos para cada cliente a lo largo del tiempo
temp_df = df_capstone_clean_2.groupby(['pk_cid', 'entry_channel'])[products_list].sum().reset_index()

# Paso 2: Identificar el producto más contratado (sumado a través de todas las particiones) por canal de entrada
# Esto es un poco más complejo, ya que ahora necesitamos considerar la suma de productos por cliente y por canal

# Calcular el producto más contratado por cliente primero
temp_df['most_contracted_product'] = temp_df[products_list].idxmax(axis=1)

# Ahora, para reflejar esto a nivel de canal de entrada, podríamos hacer una agregación adicional o mapeo directo
# Aquí elegimos mapear directamente esta información a df_capstone_clean_2, considerando la última partición conocida o cualquier otra lógica específica

# Mapear el 'most_contracted_product' de temp_df a df_capstone_clean_2
# Esto requiere un mapeo basado en 'pk_cid' y 'entry_channel'
mapping = temp_df.set_index(['pk_cid', 'entry_channel'])['most_contracted_product'].to_dict()
df_capstone_clean_2['entry_channel_product'] = df_capstone_clean_2.set_index(['pk_cid', 'entry_channel']).index.map(mapping.get).values

# Verificar los resultados
print(df_capstone_clean_2[['pk_cid', 'entry_channel', 'entry_channel_product']].sample(50))

          pk_cid entry_channel entry_channel_product
708103   1072541           KHE             em_acount
466189   1061932           KHE             em_acount
1194841  1263601           KFC           emc_account
2697613  1028210           KHE             em_acount
4458630  1450794           KHQ             em_acount
2222193  1203570           KHE             em_acount
440774   1114366           KFC             em_acount
1348894  1300256           KFC             em_acount
2740693  1420001           KHM             em_acount
581796   1336775           RED             em_acount
5839469  1022191           KHE             em_acount
2094664  1206959           KHE    short_term_deposit
5881137  1246861           KAT       payroll_account
1280761  1131818           KFC             em_acount
4188488  1030221           KHE             em_acount
1943997  1399779           KHM             em_acount
854818   1289590           KHE             em_acount
3266994  1081461           KHE    short_term_d

In [88]:
df_capstone_clean_2['entry_channel_product'].value_counts()

em_acount             4159582
short_term_deposit     981206
emc_account            227887
debit_card             203550
payroll_account        199885
payroll                 69349
long_term_deposit       61753
credit_card             31197
securities              13394
funds                   11959
pension_plan             2688
mortgage                  236
loans                     221
em_account_p               17
Name: entry_channel_product, dtype: int64

In [89]:
df_capstone_clean_2['entry_channel_product'].isnull().sum()

0

### ""channel_activity_level""

In [90]:
# CHANNEL & ACTIVITY LEVEL ASSOCIATION
# "channel_activity_level"
# Una asociación entre el canal de entrada y el nivel de actividad (active_customer) del cliente, para ver si algunos canales atraen a clientes más activos.

# Calcular la media de 'active_customer' para cada 'entry_channel'
channel_activity_mean = df_capstone_clean_2.groupby('entry_channel')['active_customer'].mean()

# Mapear los valores medios calculados de vuelta a df_capstone_clean_2
df_capstone_clean_2['channel_activity_level'] = df_capstone_clean_2['entry_channel'].map(channel_activity_mean)

# Verificar los resultados
print(df_capstone_clean_2[['pk_cid', 'entry_channel', 'channel_activity_level']].sample(10))

          pk_cid entry_channel  channel_activity_level
3425234  1062602           KHE                0.253152
4627974  1154874           KAT                0.667064
2061944  1046325           KHE                0.253152
4094961  1215478           KHE                0.253152
2526325  1440345           KHN                0.726012
3386640  1028269           KHE                0.253152
3559376  1107139           KHE                0.253152
3089219  1251677           KFC                0.636352
2365323  1121162           KAT                0.667064
4301900  1209258           KHE                0.253152


In [91]:
df_capstone_clean_2['channel_activity_level'].value_counts()

0.253152    3116090
0.636352     898070
0.302375     666987
0.667064     416522
0.639291     231300
0.635886     205938
0.726012     122396
0.858417      79353
0.292899      75910
0.870178      60899
0.705206      41775
0.294260      20662
0.759226       9349
0.661639       5636
0.747949       5241
0.755716       1662
0.292526        776
0.185440        728
0.920583        617
0.801008        397
0.746269        335
0.708197        305
1.000000        202
0.848485        198
0.663212        193
0.000000        178
0.522293        157
0.671141        149
0.634146        123
0.431579         95
0.659341         91
0.561798         89
0.553846         65
0.500000         52
0.166667         48
0.812500         48
0.829787         47
0.697674         43
0.425000         40
0.363636         33
0.652174         23
0.809524         21
0.800000         20
0.941176         17
0.411765         17
0.625000         16
0.909091         11
Name: channel_activity_level, dtype: int64

In [92]:
df_capstone_clean_2['channel_activity_level'].isnull().sum()

0

### "date_last_purchase"

In [93]:
df_capstone_clean_2["nr_products"] = df_capstone_clean_2[products_list].apply(np.sum, axis = 1)


In [94]:
df_capstone_clean_2["nr_product_trend"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_products"].transform(lambda ser : ser.diff())

In [95]:
df_capstone_clean_2["months_from_last_purchase"] = df_capstone_clean_2.groupby(["pk_cid"])["pk_partition"].cumcount()
df_capstone_clean_2.loc[df_capstone_clean_2["nr_product_trend"]>0,"date_last_purchase"]=df_capstone_clean_2.loc[df_capstone_clean_2["nr_product_trend"]>0,"pk_partition"]

In [96]:
df_capstone_clean_2["date_last_purchase"] = np.where((df_capstone_clean_2["months_from_last_purchase"]<1),df_capstone_clean_2["pk_partition"],df_capstone_clean_2["date_last_purchase"])

In [97]:
df_capstone_clean_2["date_last_purchase"] = df_capstone_clean_2.groupby(["pk_cid"])["date_last_purchase"].transform(lambda ser: ser.ffill()) 

In [98]:
df_capstone_clean_2['date_last_purchase'].value_counts()

2018-01-28    3515057
2018-07-28     900529
2018-10-28     270452
2018-09-28     232684
2018-08-28     188788
2018-11-28     146760
2018-02-28      92111
2018-03-28      91014
2018-12-28      85107
2018-06-28      75252
2018-04-28      74993
2018-05-28      71565
2019-02-28      64508
2019-01-28      62482
2019-03-28      46335
2019-04-28      28981
2019-05-28      16306
Name: date_last_purchase, dtype: int64

In [99]:
df_capstone_clean_2['date_last_purchase'].isnull().sum()

0

In [100]:
df_capstone_clean_2['months_from_last_purchase'].value_counts()

0     456373
1     447559
2     441578
3     435678
4     429478
5     422544
6     414723
7     399365
8     372030
9     349251
10    335060
11    252969
12    249273
13    245308
14    241442
15    237319
16    232974
Name: months_from_last_purchase, dtype: int64

In [101]:
df_capstone_clean_2['months_from_last_purchase'].isnull().sum()

0

In [102]:
df_capstone_clean_2['nr_product_trend'].value_counts()

 0.0    5213901
 1.0     126141
-1.0     101818
 2.0      31807
-2.0      25426
 3.0       4613
-3.0       2428
 4.0        301
-4.0         92
 5.0         18
-5.0          6
Name: nr_product_trend, dtype: int64

In [103]:
df_capstone_clean_2["nr_product_trend"].fillna(0, inplace=True)


In [104]:
df_capstone_clean_2['nr_product_trend'].isnull().sum()

0

### "customer_lifetime_value"

In [105]:
list_products = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'em_account_pp', 'credit_card',
                  'payroll', 'pension_plan', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']

#Agrupación de productos por naturaleza 

list_debt = ['loans', 'mortgage']
list_investments = ['short_term_deposit', 'funds', 'securities', 'long_term_deposit', "pension_plan"]
list_account = ['em_account_pp', 'emc_account', 'em_account_p',  'em_acount', "payroll_account","payroll"]
list_cards = ['credit_card', 'debit_card']

#relaciones de debitos o indicativos de fidelización
list_fidel = ['payroll', 'payroll_account','loans', 'mortgage', "pension_plan" ]

list_financing_60 = list_debt+list_cards
list_invest_savings_40 = list_investments
list_account_10=list_account

#Verificación
len(list_debt + list_investments + list_account + list_cards )==len(list_products)

True

In [106]:
#df_products_seg["nr_product"] = df_products_seg.apply(lambda x: sum(x[3:]), axis = 1)
df_capstone_clean_2["nr_products"] = df_capstone_clean_2[list_products].apply(np.sum, axis = 1)
df_capstone_clean_2["Max_products"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_products"].transform(np.max)

df_capstone_clean_2["nr_financing_60"] = df_capstone_clean_2[list_financing_60].apply(np.sum, axis = 1)
df_capstone_clean_2["nr_invest_savings_40"] = df_capstone_clean_2[list_invest_savings_40].apply(np.sum, axis = 1)
df_capstone_clean_2["nr_account_10"] = df_capstone_clean_2[list_account_10].apply(np.sum, axis = 1)

In [107]:
df_capstone_clean_2.sort_values(by=["pk_cid","pk_partition"],inplace=True)

In [108]:

df_capstone_clean_2["nr_financing_60_trend"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_financing_60"].transform(lambda ser : ser.diff())
df_capstone_clean_2["nr_invest_savings_40_trend"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_invest_savings_40"].transform(lambda ser : ser.diff())
df_capstone_clean_2["nr_account_10_trend"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_account_10"].transform(lambda ser : ser.diff())

In [109]:
## customer_lifetime_value_partition : aporta suma de los productos contratados o cancelados y su respectivo valor(ganancia/prejuicio)
## para una partición
## customer_lifetime_value_cumsum: : aporta suma cumulada de los productos contratados o cancelados y su 
## respectivo valor(ganancia/prejuicio)	hasta cada partición

dict_profit_category = {"financing":60, "invest":40, "account":10}

df_capstone_clean_2["customer_lifetime_value_partition"] =  df_capstone_clean_2["nr_financing_60_trend"]*dict_profit_category["financing"] + \
                                            df_capstone_clean_2["nr_invest_savings_40_trend"]*dict_profit_category["invest"] +\
                                            df_capstone_clean_2["nr_account_10_trend"]*dict_profit_category["account"]

In [110]:
condition_fillna= df_capstone_clean_2["nr_financing_60"]*dict_profit_category["financing"] +\
                  df_capstone_clean_2["nr_invest_savings_40"]*dict_profit_category["invest"] +\
                  df_capstone_clean_2["nr_account_10"]*dict_profit_category["account"] 

df_capstone_clean_2["customer_lifetime_value_partition"].fillna(condition_fillna,inplace=True)

In [111]:
#Meses o cantidad de particiones del cliente
df_capstone_clean_2["customer_lifetime_value_cumsum"] = df_capstone_clean_2.groupby(["pk_cid"])["customer_lifetime_value_partition"].cumsum()

In [112]:
df_capstone_clean_2['customer_lifetime_value_partition'].value_counts()

 0.0      5332687
 10.0      330452
 60.0       78742
-60.0       58427
-10.0       36460
 50.0       35453
-50.0       24983
 70.0       19276
 40.0       10380
-40.0        7546
 20.0        5998
 120.0       4107
 110.0       3422
-30.0        2401
 80.0        2252
 130.0       1534
 30.0        1527
-110.0       1389
-70.0        1128
-20.0         983
 100.0        639
 180.0        581
 190.0        454
 140.0        360
-120.0        332
 90.0         274
-100.0        259
 170.0        254
 160.0        162
 230.0         69
-80.0          66
-90.0          59
 150.0         54
 220.0         49
 200.0         42
-130.0         28
 210.0         24
-170.0         19
 270.0         13
 240.0         13
 250.0         12
-150.0          4
 260.0          3
-160.0          2
-140.0          2
 300.0          2
-180.0          1
Name: customer_lifetime_value_partition, dtype: int64

In [113]:
df_capstone_clean_2['customer_lifetime_value_partition'].isnull().sum()

0

In [114]:
df_capstone_clean_2['customer_lifetime_value_cumsum'].value_counts()

10.0     3949487
0.0      1121507
70.0      354477
20.0      109734
60.0      101185
120.0      85421
50.0       70420
80.0       42557
130.0      36288
40.0       25463
110.0      13874
180.0      12988
190.0       9406
140.0       7717
170.0       4491
90.0        4474
100.0       4119
160.0       3291
230.0       1244
150.0       1189
200.0        991
220.0        917
210.0        569
240.0        313
270.0        290
250.0        174
30.0         152
260.0        101
310.0         30
300.0         26
280.0         23
290.0          5
320.0          1
Name: customer_lifetime_value_cumsum, dtype: int64

In [115]:
df_capstone_clean_2['customer_lifetime_value_cumsum'].isnull().sum()

0

In [116]:
df_capstone_clean_2["nr_financing_60_trend"].fillna(0, inplace=True)
df_capstone_clean_2["nr_invest_savings_40_trend"].fillna(0, inplace=True)
df_capstone_clean_2["nr_account_10_trend"].fillna(0, inplace=True)

In [117]:
df_capstone_clean_2['nr_financing_60'].value_counts()

0    5375073
1     541384
2      46159
3        308
Name: nr_financing_60, dtype: int64

In [118]:
df_capstone_clean_2['nr_financing_60'].isnull().sum()

0

In [119]:
df_capstone_clean_2['nr_invest_savings_40'].value_counts()

0    5608901
1     333841
2      18450
3       1581
4        151
Name: nr_invest_savings_40, dtype: int64

In [120]:
df_capstone_clean_2['nr_invest_savings_40'].isnull().sum()

0

In [121]:
df_capstone_clean_2['nr_account_10'].value_counts()

1    4397533
0    1169701
2     343726
3      49037
4       2927
Name: nr_account_10, dtype: int64

In [122]:
df_capstone_clean_2['nr_account_10'].isnull().sum()

0

In [123]:
df_capstone_clean_2['nr_financing_60_trend'].value_counts()

 0.0    5815769
 1.0      82371
-1.0      63898
 2.0        542
-2.0        344
Name: nr_financing_60_trend, dtype: int64

In [124]:
df_capstone_clean_2['nr_financing_60_trend'].isnull().sum()

0

In [125]:
df_capstone_clean_2['nr_invest_savings_40_trend'].value_counts()

 0.0    5880701
 1.0      44157
-1.0      37695
 2.0        271
-2.0         91
 3.0          8
-3.0          1
Name: nr_invest_savings_40_trend, dtype: int64

In [126]:
df_capstone_clean_2['nr_invest_savings_40_trend'].isnull().sum()

0

### "saving_behavior"

In [127]:
##Agrupación de productos con naturaleza de ahorro
list_savings = ['short_term_deposit', 'funds', 'securities', 'long_term_deposit', "pension_plan"]

#df_easytrends_savings = df_capstone_clean_2

## variable "saving_behavior" contiene la suma de productos con naturaleza de ahorro
df_capstone_clean_2["saving_behavior"] = df_capstone_clean_2[list_savings].sum(axis=1)
df_capstone_clean_2["saving_behavior"].value_counts()

0    5608901
1     333841
2      18450
3       1581
4        151
Name: saving_behavior, dtype: int64

In [128]:
df_capstone_clean_2['saving_behavior'].value_counts()

0    5608901
1     333841
2      18450
3       1581
4        151
Name: saving_behavior, dtype: int64

In [129]:
df_capstone_clean_2['saving_behavior'].isnull().sum()

0

### "risk_tolerance_index"

Critérios de clasificación:

- Short-Term Deposit: Típicamente considerados de bajo riesgo ya que son inversiones a corto plazo con tasas de interés fijas y baja volatilidad.

- Loans: El riesgo asociado con los préstamos puede variar dependiendo del tipo de préstamo (por ejemplo, préstamos personales, préstamos comerciales). Generalmente, los préstamos tienen un riesgo moderado a alto dependiendo de factores como la solvencia de los prestatarios, las condiciones económicas y los términos del préstamo.

- Mortgage: Las hipotecas suelen estar garantizadas por bienes raíces, lo que las convierte en un riesgo más bajo en comparación con los préstamos no garantizados. Sin embargo, el riesgo puede variar según factores como la relación préstamo-valor, la solvencia del prestatario y las fluctuaciones en los mercados inmobiliarios.

- Funds: El riesgo asociado con los fondos puede variar ampliamente dependiendo de los activos subyacentes mantenidos por el fondo. Los fondos de renta variable son generalmente de mayor riesgo en comparación con los fondos de renta fija o del mercado monetario.

- Securities: Los valores pueden incluir una amplia gama de instrumentos financieros como acciones, bonos y derivados. El riesgo asociado con los valores depende de factores como la volatilidad del mercado, la solvencia del emisor y los objetivos de inversión

- Long-Term Deposit: Similar a los depósitos a corto plazo, generalmente se consideran de bajo riesgo ya que ofrecen tasas de interés fijas y son menos susceptibles a las fluctuaciones del mercado a corto plazo.

- Credit Card: Las tarjetas de crédito suelen tener tasas de interés altas y pueden representar un riesgo de acumulación de deuda si no se manejan de manera responsable. Sin embargo, desde la perspectiva del banco, las tarjetas de crédito también pueden ser rentables si los clientes mantienen saldos y realizan pagos oportunos.

-Payroll: Las cuentas de nómina suelen considerarse de bajo riesgo ya que implican depósitos regulares de los sistemas de nómina de los empleadores. Sin embargo, el riesgo puede variar según factores como la estabilidad del empleador y los riesgos de fraude potenciales.

- Payroll Account: Similar a las cuentas de nómina, estas son generalmente cuentas de bajo riesgo ya que implican depósitos regulares de los empleadores.

- Pension Plan: Generalmente son inversiones a largo plazo y suelen considerarse de bajo a moderado riesgo, dependiendo de la estrategia de inversión y la salud financiera del fondo de pensiones.

- Debit Card: Las tarjetas de débito suelen ser productos de bajo riesgo ya que permiten a los clientes gastar solo los fondos disponibles en sus cuentas bancarias.

- EM_Account, EM_Account_P, EM_Account_PP: Una cuenta corriente ordinaria generalmente se consideraría de riesgo bajo

- EMC_Account: Cuenta de cryptomoneda podría considerarse con un grado de riesgo alto por algunas razones: volatilidad de Mercado, falta de regulación, falta de protecciones del consumidor, posibilidad de perder acesso.


In [130]:
list_risk_low = ["short_term_deposit", "pension_plan", "long_term_deposit","payroll", "payroll_account","em_account_pp", "debit_card","em_account_p", "em_acount"]
list_risk_medium= ["mortgage"]
list_risk_high= ["loans", "funds", "securities", "credit_card", "emc_account"]

#df_easytrends_risks = df_capstone_clean_2

#Suma de productos por categoria de riesgo
df_capstone_clean_2["nr_risk_low"] = df_capstone_clean_2[list_risk_low].sum(axis=1)
df_capstone_clean_2["nr_risk_medium"] = df_capstone_clean_2[list_risk_medium].sum(axis=1)
df_capstone_clean_2["nr_risk_high"] = df_capstone_clean_2[list_risk_high].sum(axis=1)

#Porcentaje de productos por categoria de riesgo
df_capstone_clean_2["percent_risk_low"] = df_capstone_clean_2["nr_risk_low"]/len(list_risk_low)
df_capstone_clean_2["percent_risk_medium"] = df_capstone_clean_2["nr_risk_medium"]/len(list_risk_medium)
df_capstone_clean_2["percent_risk_high"] = df_capstone_clean_2["nr_risk_high"]/len(list_risk_high)


In [131]:
## Index absoluto de grado de riesgo
## Escala numerica (para evitar transformación, ya que seria ordinal): 
## 0: ninguno, 1: bajo, 2:medio, 3:alto
df_capstone_clean_2["risk_index_absolute"] = np.where(df_capstone_clean_2["nr_risk_high"]>0, 3,\
    np.where(df_capstone_clean_2["nr_risk_medium"]>0, 2, \
        np.where(df_capstone_clean_2["nr_risk_low"]>0, 1, 0)
    )
)
#df_easytrends_risks["risk_index_absolute"] = np.where(df_easytrends_risks["nr_risk_high"]>0, "Alto", df_easytrends_risks["risk_index_absolute"])

In [132]:
df_capstone_clean_2['risk_index_absolute'].value_counts()

1    4446710
0    1121507
3     394547
2        160
Name: risk_index_absolute, dtype: int64

In [133]:
df_capstone_clean_2['risk_index_absolute'].isnull().sum()

0

In [134]:
df_capstone_clean_2['percent_risk_low'].value_counts()

0.111111    4061788
0.000000    1208424
0.222222     470080
0.444444     121208
0.333333      90249
0.555556      10910
0.666667        265
Name: percent_risk_low, dtype: int64

In [135]:
df_capstone_clean_2['percent_risk_low'].isnull().sum()

0

In [136]:
df_capstone_clean_2['percent_risk_medium'].value_counts()

0.0    5962600
1.0        324
Name: percent_risk_medium, dtype: int64

In [137]:
df_capstone_clean_2['percent_risk_medium'].isnull().sum()

0

In [138]:
df_capstone_clean_2['percent_risk_high'].value_counts()

0.0    5568377
0.2     352247
0.4      38820
0.6       3280
0.8        200
Name: percent_risk_high, dtype: int64

In [139]:
df_capstone_clean_2['percent_risk_high'].isnull().sum()

0

### "life_stage_indicator"

In [140]:
#Por edad
bins = [0, 18, 35, 65, 80, 999]
labels = ['Joven', 'Adulto Joven', 'Adulto', 'Anciano', 'Anciano+']
df_capstone_clean_2['life_stage_indicator'] = pd.cut(df_capstone_clean_2['age'], bins=bins, labels=labels, right=False)

In [141]:
df_capstone_clean_2['life_stage_indicator']

1479563          Adulto
2168122          Adulto
2962973          Adulto
3628236          Adulto
4028169          Adulto
               ...     
5679916          Adulto
5679915    Adulto Joven
5679914    Adulto Joven
5679913          Adulto
5679912          Adulto
Name: life_stage_indicator, Length: 5962924, dtype: category
Categories (5, object): ['Joven' < 'Adulto Joven' < 'Adulto' < 'Anciano' < 'Anciano+']

In [142]:
df_capstone_clean_2['life_stage_indicator'].value_counts()

Adulto Joven    4478423
Adulto          1316367
Anciano           99569
Joven             35873
Anciano+          32692
Name: life_stage_indicator, dtype: int64

In [143]:
df_capstone_clean_2['life_stage_indicator'].isnull().sum()

0

In [144]:
df_capstone_clean_2.info(all)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 1479563 to 5679912
Data columns (total 79 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   pk_cid                             int64         
 1   pk_partition                       datetime64[ns]
 2   entry_date                         datetime64[ns]
 3   entry_channel                      object        
 4   active_customer                    int64         
 5   segment                            object        
 6   short_term_deposit                 int64         
 7   loans                              int64         
 8   mortgage                           int64         
 9   funds                              int64         
 10  securities                         int64         
 11  long_term_deposit                  int64         
 12  em_account_pp                      int64         
 13  credit_card                        int64         
 

In [145]:
# drops
df_capstone_clean_2.drop('nr_products', axis=1, inplace=True)
df_capstone_clean_2.drop('Max_products', axis=1, inplace=True)

#### Active_customer_rate

In [146]:
#active_customer_rate - % del tiempo como active_customers
df_capstone_clean_2["xx"] = df_capstone_clean_2.groupby(["pk_cid"])["pk_partition"].cumcount()+1
df_capstone_clean_2["yy"] = df_capstone_clean_2.groupby(["pk_cid"])["active_customer"].cumsum()
df_capstone_clean_2["active_customer_rate"] = df_capstone_clean_2["yy"]/df_capstone_clean_2["xx"]


#### "months_from_last_purchase_digital" Y "date_last_purchase_digital"

In [147]:
#Meses o cantidad de particiones del cliente
df_capstone_clean_2["months_from_last_purchase_digital"] = df_capstone_clean_2.groupby(["pk_cid"])["pk_partition"].cumcount()

#llena con la fecha de la partición de la compra
cond=((df_capstone_clean_2["nr_product_trend"]>0) & (df_capstone_clean_2["active_customer"]>0))
df_capstone_clean_2.loc[cond,"date_last_purchase_digital"]=df_capstone_clean_2.loc[cond,"pk_partition"]

#llena solo con la fecha de la primera partición
df_capstone_clean_2["date_last_purchase_digital"] = np.where((df_capstone_clean_2["months_from_last_purchase_digital"]<1),df_capstone_clean_2["pk_partition"],df_capstone_clean_2["date_last_purchase_digital"]  )

#llena nulos entre fechas con la la ultima fecha disponible
df_capstone_clean_2["date_last_purchase_digital"] = df_capstone_clean_2.groupby(["pk_cid"])["date_last_purchase_digital"].transform(lambda ser: ser.ffill()) 

#Meses desde la ultima compra
df_capstone_clean_2["months_from_last_purchase_digital"] = df_capstone_clean_2["pk_partition"].dt.to_period('M').astype(int) - df_capstone_clean_2["date_last_purchase_digital"].dt.to_period('M').astype(int) 

#### "total_products_ratio"

In [148]:
list_all_products= ['short_term_deposit', 'loans', 'mortgage', 'funds',
       'securities', 'long_term_deposit', 'em_account_pp', 'credit_card',
       'payroll', 'pension_plan', 'payroll_account', 'emc_account',
       'debit_card', 'em_account_p', 'em_acount']

total_products = len(list_all_products)
df_capstone_clean_2["total_products_ratio"] = df_capstone_clean_2["nr_product"]/total_products

In [149]:
df_capstone_clean_2.drop(columns=["xx","yy"], inplace=True)

#### "regions_cluster" - agrupación de regions_ca_id en 5 grupos basado en la población de la ultima partición 

In [150]:
df_capstone_clean_2.groupby("regions_ca_id")["salary"].count().to_frame().sort_values("salary", ascending=False)

mapeo = ["MD","AN","CT","VC","Otros" ]

df_capstone_clean_2['regions_cluster'] = df_capstone_clean_2['regions_ca_id'].apply(lambda x: x if (x in mapeo) else "Otros")

#### Nr_Transactions- numero de transaciones realizadas por el cliente (contratar o cancelar producto)

In [151]:
df_capstone_clean_2["nr_product_abs"] = df_capstone_clean_2["nr_product_trend"].abs()
df_capstone_clean_2["nr_transactions"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_product_abs"].cumsum()
df_capstone_clean_2.drop(columns=["nr_product_abs"], inplace=True)

df_capstone_clean_2["nr_transactions"].fillna(0,inplace=True)

In [152]:
df_capstone_clean_2["profit_60"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_financing_60_trend"].transform(lambda x : x[x>0].sum()*60)
df_capstone_clean_2["profit_40"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_invest_savings_40_trend"].transform(lambda x : x[x>0].sum()*40)
df_capstone_clean_2["profit_10"] = df_capstone_clean_2.groupby(["pk_cid"])["nr_account_10_trend"].transform(lambda x : x[x>0].sum()*10)
df_capstone_clean_2["total_profit_customer"] = df_capstone_clean_2["profit_60"] + df_capstone_clean_2["profit_40"] + df_capstone_clean_2["profit_10"]


In [153]:
# pd.set_option
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_info_columns', 10)
pd.set_option('display.width', 10)
pd.set_option('display.max_rows', 150)
pd.set_option('display.colheader_justify', 'left')

In [155]:
listarNulos(df_capstone_clean_2)

,Nulos,porcentaje


In [156]:
# Guardamos pickle

DATA_PATH = "../../data/processed/"
FILE_NAME = "df_capstone_newFeatures.pkl.gz"
PICKLE_PATH = DATA_PATH+FILE_NAME

df_capstone_clean_2.to_pickle(PICKLE_PATH, compression='gzip')